In [ ]:
from dotenv import load_dotenv
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_community.chat_message_histories import Neo4jChatMessageHistory
from langchain_openai import ChatOpenAI, AzureChatOpenAI
from langchain.schema import StrOutputParser
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_community.graphs import Neo4jGraph
import os
from uuid import uuid4

load_dotenv()

In [2]:
llm2 = AzureChatOpenAI(
    azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
    openai_api_type=os.getenv("AZURE_OPENAI_API_TYPE"),
    openai_api_key=os.getenv("AZURE_OPENAI_API_KEY"),
    azure_deployment=os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT_NAME"),
    openai_api_version=os.getenv("AZURE_OPENAI_API_VER")
)

In [ ]:
print(llm2)

In [ ]:
response = llm2.invoke("Hello, how are you?")
print(response)

In [ ]:
ans = response.response_metadata

print(ans)

In [ ]:
# Retrieve environment variables
NEO4J_URL = os.getenv("NEO4J_URL")
NEO4J_USERNAME = os.getenv("NEO4J_USERNAME")
NEO4J_PASSWORD = os.getenv("NEO4J_PASSWORD")
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

# Initialize Neo4j graph connection
graph = Neo4jGraph(
    url=NEO4J_URL,
    username=NEO4J_USERNAME,
    password=NEO4J_PASSWORD
)

print(graph)

In [ ]:
# Generate a unique session ID
SESSION_ID = str(uuid4())
print(f"Session ID: {SESSION_ID}")


def get_memory(session_id):
    return Neo4jChatMessageHistory(session_id=session_id, graph=graph)

In [ ]:
chat_llm = ChatOpenAI(openai_api_key=os.getenv("OPENAI_API_KEY"))

print(chat_llm)

In [6]:
SCHEMA = """
(Movie)-[:IN_GENRE]->(Genre), (Movie)-[:DIRECTED_BY]->(Director), 
(Movie)-[:HAS_ACTOR]->(Actor), (Movie)-[:HAS_RATING]->(Rating)
"""

In [7]:
CYPHER_GENERATION_TEMPLATE = f"""
You are an expert Neo4j Developer translating user questions into Cypher to answer questions about movies and provide recommendations.
Convert the user's question based on the schema.

Instructions:
Use only the provided relationship types and properties in the schema.
For movie titles that begin with "The", move "the" to the end, For example "The 39 Steps" becomes "39 Steps, The" or "The Matrix" becomes "Matrix, The".

If no data is returned, do not attempt to answer the question.
Only respond to questions that require you to construct a Cypher statement.
Do not include any explanations or apologies in your responses.

Schema: {SCHEMA}
Question: {{question}}
"""

In [8]:
few_shot_example = """
Find all movies directed by Christopher Nolan
MATCH (m:Movie)-[:DIRECTED_BY]->(d:Director {name: 'Christopher Nolan'})
RETURN m.title
"""

In [ ]:
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are an expert in movies and Neo4j queries."),
    ("system", "{context}"),
    MessagesPlaceholder(variable_name="chat_history"),
    ("human", "{question}")
])




print(prompt)

In [ ]:
# Create the chat chain with the message history and the output parser
chat_chain = prompt | chat_llm | StrOutputParser()

print(chat_chain)

In [ ]:
# Chat with message history integration
chat_with_message_history = RunnableWithMessageHistory(
    chat_chain,
    get_memory,
    input_messages_key="question",
    history_messages_key="chat_history",
)

print(chat_with_message_history)

In [16]:
question = "Find all movies directed by James Cameron"

response = chat_with_message_history.invoke(
    {
        "context": "You are an expert in movies and Neo4j queries.",
        "question": question,
    },
    config={
        "configurable": {"session_id": SESSION_ID}
    }
)

In [ ]:
print(response)